In [1]:
FILE_PATH="/Users/gmanvel/repos/rag-fast-flow/data/fast_flow.pdf"
%pip install PyMuPDF

  Using cached pymupdf-1.26.5-cp39-abi3-macosx_11_0_arm64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.5-cp39-abi3-macosx_11_0_arm64.whl (22.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fitz  # PyMuPDF
doc = fitz.open(FILE_PATH)

In [3]:
def extract(doc):
    content_structure = []  # List to maintain header order
    current_header = None
    
    # Start from page 3
    for page_num in range(2, doc.page_count):  # Starting from 2 because pages are 0-based
        page = doc[page_num]
        blocks = page.get_text("dict")["blocks"]
        
        # Collect all headers and sections for this page first
        page_headers = []
        page_sections = []
        
        for block in blocks:
            if "lines" not in block:
                continue
                
            for line in block["lines"]:
                for span in line["spans"]:
                    # Check if the text is a header (size 34) or section (size 18)
                    if span["size"] in [34, 18]:
                        text = span["text"].strip()
                        if not text:  # Skip empty text
                            continue
                            
                        if span["size"] == 34:  # This is a header
                            page_headers.append(text)
                        else:  # This is a section
                            page_sections.append(text)
        
        # Process headers for this page (if any)
        if page_headers:
            # If we have a current header, store it before starting a new one
            if current_header:
                content_structure.append(current_header)
            
            # Create new header with all header text from this page combined
            current_header = {
                'header': {
                    'text': ' '.join(page_headers),
                    'size': 34,
                    'page': page_num + 1  # Convert to 1-based page numbers
                },
                'sections': []
            }
        
        # Process sections for this page (if any)
        if page_sections:
            # If we have no header yet, create a placeholder
            if not current_header:
                current_header = {
                    'header': None,
                    'sections': []
                }
            
            # Add combined sections from this page
            current_header['sections'].append({
                'text': ' '.join(page_sections),
                'size': 18,
                'page': page_num + 1
            })
    
    # Don't forget to append the last header if we have one
    if current_header:
        content_structure.append(current_header)
    
    return content_structure

In [4]:
# Extract headers and sections
content_structure = extract(doc)

# Print the structure
for idx, section in enumerate(content_structure, 1):
    if section['header']:
        print(f"\nHeader (Page {section['header']['page']}): {section['header']['text']} (size: {section['header']['size']})")
    else:
        print("\nOrphan sections (no header):")
    
    if section['sections']:
        print("Sections:")
        for subsection in section['sections']:
            print(f"  - [Page {subsection['page']}] {subsection['text']} (size: {subsection['size']})")


Header (Page 3): Introduction (size: 34)

Header (Page 5): Building Adaptive Systems for a Fast Flow of Change (size: 34)
Sections:
  - [Page 5] The business strategy perspective with Wardley Mapping (size: 18)
  - [Page 7] Visualizing the landscape - the Wardley Map (size: 18)
  - [Page 9] The Domain-Driven Design perspective (size: 18)
  - [Page 11] Discovering subdomains and determining their evolution stages (size: 18)
  - [Page 12] Domain Models and Bounded Contexts (size: 18)
  - [Page 14] Strategic design and Wardley’s Doctrine (size: 18)
  - [Page 15] The team organization perspective with Team Topologies (size: 18)
  - [Page 17] Architecture for flow (size: 18)
  - [Page 18] Team Topologies and Wardley’s Doctrine (size: 18)

Header (Page 19): About Team Topologies and Context Mapping (size: 34)
Sections:
  - [Page 19] The problem space is mainly uncharted territory (size: 18)
  - [Page 20] Have you been Spotified? Team Topologies categorizations (size: 18)
  - [Page 21] Three